In [1]:
### header ###
__author__ = "Hunter Bennett"
__license__ = "BSD"
__email__ = "hunter.r.bennett@gmail.com"
%load_ext autoreload
%autoreload 2
### imports ###
import sys
%matplotlib inline
import os
import re
import pandas as pd
import numpy as np
import matplotlib
import threading
import matplotlib.pyplot as plt 
import seaborn as sns
from collections import Counter
matplotlib.rcParams['savefig.dpi'] = 200
sys.setrecursionlimit(3000)
import pickle
from sklearn import preprocessing
import time
sns.set(font_scale=1)
sns.set_context('talk')
sns.set_style('white')

# import custom functions
import sys
sys.path.insert(0, '/home/h1bennet/code/')
from hbUtils import ngs_qc, quantile_normalize_df
from plotting_scripts import label_point, pca_rpkm_mat, get_diff_volcano

In [2]:
workingDirectory = '/home/h1bennet/lxr/results/181030_dmhca_ip/'
if not os.path.isdir(workingDirectory):
    os.mkdir(workingDirectory)
os.chdir(workingDirectory)


### Drop 12B based on PCA analysis in 181030_DMHCA_IP_RNA_Summary.ipynb

In [13]:
raw = pd.read_csv('./expression/rna_exp_raw.tsv', sep='\t')
raw.drop(axis=1, labels=['/home/h1bennet/lxr/data/mouse/RNA/dmhca_ip/C57B6J_Tim4PosKupffer_RNA_Veh_12h_C5712B_HBENN_l20181012_TTAGGC (13266091.0 total)'],
         inplace=True)
raw.to_csv('./expression/rna_exp_raw_dropped.tsv', sep='\t', index=False)

In [15]:
%%bash
source activate deseq
getDiffExpression.pl ./expression/rna_exp_raw_dropped.tsv mg_dmhca mg_dmhca mg_dmhca mg_dmhca mg_veh mg_veh mg_veh kupffer_dmhca kupffer_dmhca kupffer_dmhca kupffer_dmhca kupffer_veh kupffer_veh -AvsA > ./expression/diff_output.txt




	Differential Expression Program: DESeq2
	Autodetecting input file format...
	Using DESeq2 to calculate differential expression/enrichment...
	Autodetected analyzeRepeats.pl file

	Performing variance stabalization (rlog)...

	Output Stats mg_dmhca vs. mg_veh:
		Total Genes: 24537
		Total Up-regulated in mg_veh vs. mg_dmhca: 0 (0.000%) [log2fold>1, FDR<0.05]
		Total Dn-regulated in mg_veh vs. mg_dmhca: 0 (0.000%) [log2fold<-1, FDR<0.05]
	Output Stats mg_dmhca vs. kupffer_dmhca:
		Total Genes: 24537
		Total Up-regulated in kupffer_dmhca vs. mg_dmhca: 3285 (13.388%) [log2fold>1, FDR<0.05]
		Total Dn-regulated in kupffer_dmhca vs. mg_dmhca: 2450 (9.985%) [log2fold<-1, FDR<0.05]
	Output Stats mg_dmhca vs. kupffer_veh:
		Total Genes: 24537
		Total Up-regulated in kupffer_veh vs. mg_dmhca: 2607 (10.625%) [log2fold>1, FDR<0.05]
		Total Dn-regulated in kupffer_veh vs. mg_dmhca: 1932 (7.874%) [log2fold<-1, FDR<0.05]
	Output Stats mg_veh vs. kupffer_dmhca:
		Total Genes: 24537
		Total Up-regu

11C doesn't appear to be responding as well as the other three samples, lets try eliminating it bc maybe the injection didn't work as well?

In [28]:
raw = pd.read_csv('./expression/rna_exp_raw.tsv', sep='\t')
raw.drop(axis=1, labels=['/home/h1bennet/lxr/data/mouse/RNA/dmhca_ip/C57B6J_Tim4PosKupffer_RNA_Veh_12h_C5712B_HBENN_l20181012_TTAGGC (13266091.0 total)',
                         '/home/h1bennet/lxr/data/mouse/RNA/dmhca_ip/C57B6J_Tim4PosKupffer_RNA_DMHCA50mgperkg_12h_C5711C_HBENN_l20181012_ACAGTG (12321565.0 total)'],
         inplace=True)
raw.to_csv('./expression/rna_exp_raw_dropped_11C_12B.tsv', sep='\t', index=False)

In [30]:
%%bash
source activate deseq
getDiffExpression.pl ./expression/rna_exp_raw_dropped_11C_12B.tsv mg_dmhca mg_dmhca mg_dmhca mg_dmhca mg_veh mg_veh mg_veh kupffer_dmhca kupffer_dmhca kupffer_dmhca kupffer_veh kupffer_veh -AvsA > ./expression/diff_output_drop11c.txt




	Differential Expression Program: DESeq2
	Autodetecting input file format...
	Using DESeq2 to calculate differential expression/enrichment...
	Autodetected analyzeRepeats.pl file

	Performing variance stabalization (rlog)...

	Output Stats mg_dmhca vs. mg_veh:
		Total Genes: 24537
		Total Up-regulated in mg_veh vs. mg_dmhca: 0 (0.000%) [log2fold>1, FDR<0.05]
		Total Dn-regulated in mg_veh vs. mg_dmhca: 0 (0.000%) [log2fold<-1, FDR<0.05]
	Output Stats mg_dmhca vs. kupffer_dmhca:
		Total Genes: 24537
		Total Up-regulated in kupffer_dmhca vs. mg_dmhca: 3157 (12.866%) [log2fold>1, FDR<0.05]
		Total Dn-regulated in kupffer_dmhca vs. mg_dmhca: 2531 (10.315%) [log2fold<-1, FDR<0.05]
	Output Stats mg_dmhca vs. kupffer_veh:
		Total Genes: 24537
		Total Up-regulated in kupffer_veh vs. mg_dmhca: 2607 (10.625%) [log2fold>1, FDR<0.05]
		Total Dn-regulated in kupffer_veh vs. mg_dmhca: 1932 (7.874%) [log2fold<-1, FDR<0.05]
	Output Stats mg_veh vs. kupffer_dmhca:
		Total Genes: 24537
		Total Up-reg

### Differential gene analysis

In [31]:
# import differential gene expression
diff_gene = pd.read_csv('./expression/diff_output_drop11c.txt', sep='\t', index_col=0)
diff_gene.index.rename('RepeatID', inplace=True)
mm10_gene = diff_gene['Annotation/Divergence'].str.split('|').str[0]

# create gene name index
diff_gene['gene'] = mm10_gene
diff_gene = diff_gene.reset_index().set_index('gene').drop(labels='RepeatID', axis=1)

# filter entries without a gene
# diff_gene = diff_gene.loc[diff_gene.index.notna(), :]
print(diff_gene.shape)

(24537, 37)


In [32]:
def pull_comparision(diff_gene, comp):
    return diff_gene.loc[:, [comp + ' Log2 Fold Change',
                            comp + ' p-value',
                            comp + ' adj. p-value']]

# extract groups
comp_dict = {}
pattern='(\w* vs. \w*).*'
for col in diff_gene.columns.values:
    m = re.search(string=col, pattern=pattern)
    if m:
        df = pull_comparision(diff_gene, m.group(1))
        df.columns = ['log2fc', 'pval', 'adj_pval']
        comp_dict[re.sub('G0[0-9]_', '', m.group(1))] = df

In [33]:
de = comp_dict['kupffer_dmhca vs. kupffer_veh']
de = de.sort_values('adj_pval', ascending=True)

In [40]:
de.loc[(de.log2fc < 0) & (de.adj_pval <= 0.05)].index.tolist()

['Jdp2',
 'Pik3r5',
 'Fpr2',
 'Slc7a1',
 'Cd14',
 'Cpd',
 'Rapgef2',
 'Mllt6',
 'Sod2',
 'Ddhd1',
 'Icosl',
 'Rn45s',
 'Rapgef5',
 'Fpr1',
 'Nfkb2',
 'Siglece',
 'Icam1',
 'Tnip1',
 'Mthfd2',
 'Relb',
 'Clec4d',
 'Rnf103',
 'Sh3bgrl2',
 'Abcc1',
 'Igsf6',
 'Ngp',
 'Cd82',
 'Rap2a',
 'Mir6236',
 'Slc31a2',
 'Dram1',
 'Tmem123',
 'Birc3',
 'Dst',
 'Sla',
 'Notch1',
 'Ctla2b',
 'Zc3h12a',
 'Phldb1',
 'Mesdc1',
 'Ctnnd1',
 'Vasp',
 'N4bp1',
 'Ppm1n',
 'Nfkb1',
 'Clec4a1',
 'Abtb2',
 'Fnbp1l',
 'Spic',
 'Nadk',
 'Cd300e',
 'Rnd1',
 'Hs3st3b1',
 'Runx1',
 'Fth1',
 'Dusp16',
 'Hivep3',
 'Slc11a2',
 'Samsn1',
 'Rassf4',
 'Cd274',
 'Zmynd15',
 'Klf9',
 'Prex1',
 'Acsl5',
 'Flna',
 'Lars2',
 'Pot1b',
 'Aoah',
 'Hpn',
 'Rhoc',
 'Itpkb',
 'Mmd',
 'Nab1',
 'Slc16a10',
 'Fam49a',
 'Arl5c',
 'Prdx5',
 'Cdc42ep2',
 'Klf7',
 'Stard7',
 'Csf3r',
 'Irak2',
 'Tnfaip2',
 'Pou2f2',
 'Gramd1a',
 'Cxcl16',
 'Cfb',
 'Gsap',
 'Lpar1',
 'Stk40',
 'Clec4a2',
 'Hivep2',
 'Gas7',
 'Vim',
 'Slc31a1',
 'Dmxl2',
 'Myo

In [37]:
de.loc[(de.log2fc > 0) & (de.adj_pval <= 0.05)]

log2fc          pval      adj_pval
gene                                           
Chp2       2.530173  1.563169e-20  2.592405e-17
Gm1966     1.468507  1.342226e-18  1.038794e-15
Icos       2.129204  5.704771e-17  3.311334e-14
Ccr5       1.002861  1.541458e-14  5.592122e-12
Arl4c      1.348384  1.230126e-13  3.661674e-11
Fam167a    1.570755  1.429324e-13  3.858842e-11
Fam102a    1.341840  1.077033e-12  2.404477e-10
Slc46a3    1.658746  2.564297e-12  5.512764e-10
Scd1       1.047097  4.544971e-12  9.256591e-10
Hpgd       1.658006  6.869697e-12  1.329172e-09
Tgfbi      1.037099  1.000553e-11  1.814910e-09
Rnf144b    1.840820  2.298825e-11  3.812437e-09
Tmtc1      2.169827  2.705637e-11  4.423908e-09
Zfp36l2    1.279812  3.113886e-11  5.020708e-09
Engase     1.427111  6.139808e-10  7.582663e-08
Klrk1      1.495583  1.333120e-09  1.517273e-07
Rnf150     1.840195  1.370214e-09  1.544351e-07
Ckb        0.979941  1.667280e-09  1.808921e-07
Jup        1.029216  1.876188e-09  1.998226e-07
Ifit2      1.798123  2.640353e-09  2.761429e-07
Dhrs3      1.150658  2.710407e-09  2.809385e-07
Mrc1       0.765444  3.265051e-09  3.295998e-07
Gprc5c     1.530753  3.255856e-09  3.295998e-07
Nrp1       1.128611  3.989789e-09  3.955139e-07
Pld1       1.397719  5.040766e-09  4.836220e-07
Snx5       0.850993  1.013318e-08  8.844821e-07
Fam46a     1.085286  1.033308e-08  8.951997e-07
Cebpa      1.387267  1.391172e-08  1.170299e-06
Clec1b     1.115681  1.862432e-08  1.522604e-06
Rab39      1.154294  1.926395e-08  1.563882e-06
...             ...           ...           ...
Kif13b     0.753140  4.700416e-03  4.344517e-02
Synrg      0.540581  4.716390e-03  4.352351e-02
Manba      0.585863  4.768203e-03  4.396670e-02
Folr2      0.530004  4.780332e-03  4.404355e-02
Tpra1      0.563434  4.831419e-03  4.444454e-02
Aim1       0.475194  4.831511e-03  4.444454e-02
Srgap3     1.013500  4.835814e-03  4.444890e-02
Nacc2      0.636237  4.886740e-03  4.481891e-02
Plxnb3     1.198055  4.949809e-03  4.522572e-02
Ift122     1.548569  4.943679e-03  4.522572e-02
Ap3m2      0.808098  4.941985e-03  4.522572e-02
Gatc       0.654940  4.957371e-03  4.524380e-02
Khnyn      0.628956  5.028910e-03  4.578871e-02
Syne2      0.622870  5.045105e-03  4.586423e-02
Plxna4os1  1.162337  5.065502e-03  4.597765e-02
Ncoa7      0.535727  5.076625e-03  4.602919e-02
Vps13a     0.548255  5.137587e-03  4.648655e-02
Epsti1     0.545807  5.148526e-03  4.651302e-02
Socs5      0.508676  5.257267e-03  4.731725e-02
Zfp148     0.530402  5.307288e-03  4.772448e-02
Zkscan1    0.780993  5.318699e-03  4.775312e-02
Snx24      0.574285  5.355943e-03  4.797619e-02
Kif19a     0.781333  5.380990e-03  4.813232e-02
Baz2a      0.418267  5.449432e-03  4.866343e-02
Zer1       0.689643  5.499443e-03  4.898082e-02
Mta3       0.636843  5.501851e-03  4.898082e-02
Mgme1      0.799318  5.590458e-03  4.945128e-02
Trim47     0.522941  5.593033e-03  4.945128e-02
Phospho2   0.838028  5.586896e-03  4.945128e-02
Mill2      2.300832  5.651557e-03  4.985480e-02

[528 rows x 3 columns]

Write gene lists and volcano plots for comparison

In [39]:
#### Write gene list for analysis with metascape
pval = 0.05 # p value cutoff
log2fc = np.log2(2) # fc cutoff for interesting genes
fc_max = 1.3 # max fc to define as unchanged
up_down_dict = {}

if not os.path.isdir('./gene_lists'):
    os.mkdir('./gene_lists')

for key, diff in comp_dict.items():
    
    up = ( comp_dict[key].adj_pval <= pval ) & \
    ( comp_dict[key].log2fc >= log2fc )
    
    dn = ( comp_dict[key].adj_pval <= pval ) & \
    ( comp_dict[key].log2fc <= -log2fc )
    
    with open('./gene_lists/'+key+'_up_log2fc%.2f_p%.2f.txt' % (log2fc, pval), 'w') as f:
        f.write('gene\n')
        for val in up[up].index.values:
            f.write(val+'\n')
        f.close()

    with open('./gene_lists/'+key+'_dn_log2fc%.2f_p%.2f.txt' % (log2fc, pval), 'w') as f:
        f.write('gene\n')
        for val in dn[dn].index.values:
            f.write(val+'\n')
        f.close()
        
    up_down_dict[key[12:]] = ( len(dn[dn].index.values),
                               len(up[up].index.values) )
    
    # make volcano plot
    get_diff_volcano(diff, label_n_top_genes=10, plot_size=4)
    plt.title(key)
    plt.savefig('./figures/'+key+'_volcano.pdf', bbox_inches='tight')
    plt.close()
        
#     sns.clustermap(
#         rpkm.loc[rpkm.index.isin(up[up].index.values), :].iloc[:, 7:],
#         z_score=0,
#         cmap='RdBu_r',
#         col_cluster=False)
#     plt.savefig('./figures/'+key+'_up_fc2_p01_heatmap.pdf', bbox_inches='tight')
#     plt.close()
    
#     sns.clustermap(
#         rpkm.loc[rpkm.index.isin(dn[dn].index.values), :].iloc[:, 7:],
#         z_score=0,
#         cmap='RdBu_r',
#         col_cluster=False)
#     plt.savefig('./figures/'+key+'_dn_fc2_p01_heatmap.pdf', bbox_inches='tight')
#     plt.close()

Looking at the gene list in metascape many of these genes light up for the inflammatory response